In [1]:
import ta
import time
import math
import pprint
import asyncio
import pandas as pd
import SecretConfig
import ccxt.async_support as ccxt
from IPython.display import clear_output

# Définir l'échange
exchange = ccxt.bitget({
    "apiKey":SecretConfig.APIKey,
    "secret":SecretConfig.SecretKey,
    'password': SecretConfig.Passphrase,
    "options": {'defaultType': 'swap', 'adjustForTimeDifference' : True},
    "timeout": 60000,
    "enableRateLimit": True,
})

In [2]:
async def fetch_data():

    global full_rolling_dataframe
    global tickers_name_list
    global occurences_count
    global df_top_10
    global windparams

    futures_tickers = await exchange.fetch_tickers(params={'type': 'swap'})
    tickers_name_list = futures_tickers.keys()

    # Initialiser un dictionnaire pour stocker les nouvelles données
    data = {}

    # Parcourir chaque contrat à terme
    for ticker in tickers_name_list:

        # Récupérer les informations pertinentes pour chaque contrat à terme
        ticker_name = ticker.split('/')[0]
        data[ticker_name + ' Last Price'] = futures_tickers.get(ticker, {}).get('info', {}).get('last')
        data[ticker_name + ' Var Last Price'] = None
        data[ticker_name + ' Var Carre Last Price'] = None
        data[ticker_name + ' EMA_1'] = None
        data[ticker_name + ' Var EMA_1'] = None
        data[ticker_name + ' EMA_2'] = None
        data[ticker_name + ' Var EMA_2'] = None
        data[ticker_name + ' Diff Carre Price_EMA_1']  = None
        data[ticker_name + ' Divergence Price_EMA_1'] = None
        data[ticker_name + ' Divergence Moyen Price_EMA_1'] = None
        data[ticker_name + ' Diff Carre EMA_1_2']  = None
        data[ticker_name + ' Divergence EMA_1_2'] = None
        data[ticker_name + ' Divergence Moyen EMA_1_2'] = None
        data[ticker_name + ' Condition'] = None
            


    # Convertir le dictionnaire en DataFrame
    data['Timestamp'] = futures_tickers.get(ticker, {}).get('timestamp')
    new_data_df = pd.DataFrame([data])

    # Convertir les colonnes en numérique et définir 'Timestamp' comme index
    new_data_df = new_data_df.apply(pd.to_numeric)
    new_data_df['Timestamp'] = pd.to_datetime(new_data_df['Timestamp'], unit='ms')
    new_data_df['Timestamp'] = new_data_df['Timestamp'].dt.round('10L')
    new_data_df = new_data_df.set_index('Timestamp')

    # Ajouter les nouvelles données au DataFrame roulant
    full_rolling_dataframe = pd.concat([full_rolling_dataframe, new_data_df])

    # Calculer les indicateurs techniques
    for ticker in tickers_name_list:

        ticker_name = ticker.split('/')[0]

        if len(full_rolling_dataframe) >= wind_Var_Price + 1:

            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Var Last Price'] = (((full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Last Price'] - full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_Price], ticker_name + ' Last Price']) 
                                                                        /  full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_Price], ticker_name + ' Last Price']) * 100)

            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Var Carre Last Price'] = (((full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Last Price'] - full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_Price], ticker_name + ' Last Price']) 
                                                                        /  full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_Price], ticker_name + ' Last Price']) * 100)**2
        else :
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Var Last Price'] = None
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Var Carre Last Price'] = None
        
        if len(full_rolling_dataframe) >= wind_EMA1 :
        
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' EMA_1'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window= wind_EMA1).iloc[-1]
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Diff Carre Price_EMA_1'] = (full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Last Price'] - full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' EMA_1'])**2
            
            if len(full_rolling_dataframe) >= wind_EMA1 + 2 and full_rolling_dataframe.loc[full_rolling_dataframe.index[-2], ticker_name + ' Diff Carre Price_EMA_1'] != 0 :

                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Price_EMA_1']= ((full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Diff Carre Price_EMA_1']  - full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Div_Price_EMA1], ticker_name + ' Diff Carre Price_EMA_1']) 
                                                                        /  full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Div_Price_EMA1], ticker_name + ' Diff Carre Price_EMA_1']) * 100
            else :

                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Price_EMA_1'] = 0

            if len(full_rolling_dataframe) >= wind_EMA1 + wind_Div_Price_EMA1:
                
                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Moyen Price_EMA_1'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Divergence Price_EMA_1'], window= wind_Div_Moy_Price_EMA1).iloc[-1]

            else :
                
                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Moyen Price_EMA_1'] = None

            if len(full_rolling_dataframe) >= wind_EMA1 + wind_Var_EMA1 and full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_EMA1], ticker_name + ' EMA_1'] != 0:
            
                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Var EMA_1'] = ((full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' EMA_1'] - full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_EMA1], ticker_name + ' EMA_1'] ) 
                                                               /  full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_EMA1], ticker_name + ' EMA_1']) * 100
            
            else :
            
                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Var EMA_1'] = 0

        else :
            
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' EMA_1'] = None
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Diff Carre Price_EMA_1'] = None
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Price_EMA_1'] = None
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Moyen Price_EMA_1'] = None

        if len(full_rolling_dataframe) >= wind_EMA2:

            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' EMA_2'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window= wind_EMA2).iloc[-1]
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Diff Carre EMA_1_2'] = (full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' EMA_1'] - full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' EMA_2'])**2

            if len(full_rolling_dataframe) >= wind_EMA2 + 2 and full_rolling_dataframe.loc[full_rolling_dataframe.index[-2], ticker_name + ' Diff Carre EMA_1_2'] != 0:

                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence EMA_1_2'] = ((full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Diff Carre EMA_1_2'] - full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Div_EMA1_2], ticker_name + ' Diff Carre EMA_1_2']) 
                                                                        /  full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Div_EMA1_2], ticker_name + ' Diff Carre EMA_1_2']) * 100

            else :
                
                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence EMA_1_2'] = 0
            
            if len(full_rolling_dataframe) >= wind_EMA2 + wind_Div_EMA1_2:

                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Moyen EMA_1_2'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Divergence EMA_1_2'], window= wind_Div_Moy_EMA1_2).iloc[-1]

            else :
                
                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Moyen EMA_1_2'] = None

            if len(full_rolling_dataframe) >= wind_Var_EMA2 and full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_EMA2], ticker_name + ' EMA_2'] != 0 :

                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Var EMA_2'] = (((full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' EMA_2'] - full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_EMA2], ticker_name + ' EMA_2'] ) 
                                                               / full_rolling_dataframe.loc[full_rolling_dataframe.index[- wind_Var_EMA2], ticker_name + ' EMA_2']) * 100)
                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Condition'] = (full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Price_EMA_1'] > 0) & (full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence EMA_1_2'] > 0)
            else :
            
                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Var EMA_2'] = 0
                full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Condition'] = None

        else :
            
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' EMA_2'] = None
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Diff Carre EMA_1_2'] = None
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence EMA_1_2']= None
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Moyen EMA_1_2'] = None
            full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Condition'] = None


    # Convertir en données numériques
    full_rolling_dataframe = full_rolling_dataframe.apply(pd.to_numeric)
    full_rolling_dataframe = full_rolling_dataframe.copy()
    
    # Limiter le DataFrame roulant aux 600 dernières périodes
    # full_rolling_dataframe = full_rolling_dataframe.tail(50)

    # Obtenez la liste des ticker_names
    ticker_names = set(col.split(' ')[0] for col in full_rolling_dataframe.columns if ' Divergence Moyen Price_EMA_1' in col)

    # Sélectionnez les colonnes de variation
    cols_variation = [ticker + ' Divergence Moyen Price_EMA_1' for ticker in ticker_names if full_rolling_dataframe[ticker_name + ' Condition'].iloc[-1]]

    # Obtenez la dernière ligne du DataFrame
    last_row = full_rolling_dataframe.iloc[-1]

    # Créez une série avec seulement les colonnes de variation de la dernière ligne
    last_row_variation = last_row[cols_variation]

    # Obtenez les 10 plus grandes variations
    top_10_variations = last_row_variation.nlargest(10).keys().tolist()

    top_10_variations = [name.replace(' Divergence Moyen Price_EMA_1', '') for name in top_10_variations]

    # top_10_variations = [f"{name.replace(' Var EMA_1', '')}: {full_rolling_dataframe[name].iloc[-1]}, {full_rolling_dataframe[name.replace(' Var EMA_1', ' Divergence Moyen Wind1')].iloc[-1]}" for name in top_10_variations]

    top_10_variations += [None] * (10 - len(top_10_variations))

    # Créez un nouveau DataFrame avec seulement les 10 plus grandes variations
    if df_top_10.empty:
        df_top_10 = pd.DataFrame([top_10_variations], columns=['top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8', 'top9', 'top10'], index=[last_row.name])
    else:
        new_row = pd.DataFrame([top_10_variations], columns=['top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8', 'top9', 'top10'], index=[last_row.name])
        df_top_10 = pd.concat([df_top_10, new_row])


    df_top_10 = df_top_10.tail(10)
    occurences_count = df_top_10.stack().value_counts()
    
    # Afficher les 10 dernières lignes du DataFrame
    # clear_output(wait=True)
    # display(full_rolling_dataframe.tail(20))
    # display(df_top_10)
    # pprint.pprint(occurences_count)
    # display(len(occurences_count))
    # print(df_top_10)
    # pprint.pprint(full_rolling_dataframe[ticker_name + ' ATR'])
    # pprint.pprint(full_rolling_dataframe[ticker_name + ' ATR Change'])
    # pprint.pprint(full_rolling_dataframe)

    # Attendre 1 seconde avant de récupérer à nouveau les données
    # await exchange.close()
    await asyncio.sleep(0.01)

In [3]:
async def trad():
    
    global full_rolling_dataframe
    global df_open_trade_info
    global tickers_name_list
    global realbalance
    global occurences_count
    global nbr_open_trade 
    global trade_result
    global trail_stop
    global balance
    global engaged
    global frais
    

    if not df_open_trade_info.empty:

        for index, row in df_open_trade_info.iterrows():

            if row["Status"] == "Open":
                
                if row["Side"] == "Long":

                    real_PnL = None
                    real_Size = None
                    df_open_trade_info["PnL"] = df_open_trade_info["PnL"].apply(pd.to_numeric)
                    df_open_trade_info["Size"] = df_open_trade_info["Size"].apply(pd.to_numeric)

                    if row["Trail stop price"] < full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price'] * (1 - trail_stop):

                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trail stop price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price'] * (1 - trail_stop)
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']

                    elif row["Trail stop price"] >= full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']:

                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Status"] = "Close"
                        trade_close_price = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trade close price"] = trade_close_price
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Duration"] = (full_rolling_dataframe.index[-1] - row["Timestamp"]).round('S')
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"] = row["Size"] * (((trade_close_price* 1.0006) - row["Trade open price"]) / row["Trade open price"])
                        # df_open_trade_info["PnL"] = df_open_trade_info["PnL"].apply(pd.to_numeric)
                        real_PnL = df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"].item()
                        balance = balance + real_PnL
                        frais = frais + ((row["Size"] + real_PnL) * 0.0004) 
                        print(f'trade {row["Symbol"]} close withe PnL {real_PnL}')
                        nbr_open_trade -= 1

                    else :
                        actual = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = actual
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"] = row["Size"] * (((actual* 1.0006) - row["Trade open price"]) / row["Trade open price"])

                        if nbr_open_trade > 25 and row["PnL"] <= 0 :
                            df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trail stop price"]  = row["Trail stop price"] + ((row["Actual price"] - row["Trail stop price"]) / 3)
                else:

                    if row["Trail stop price"] > full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price'] * (1 + trail_stop):

                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trail stop price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price'] * (1 + trail_stop)
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']

                    elif row["Trail stop price"] <= full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']:
                    
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Status"] = "Close"
                        trade_close_price = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trade close price"] = trade_close_price
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Duration"] = (full_rolling_dataframe.index[-1] - row["Timestamp"]).round('S') 
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"] = row["Size"] * -(((trade_close_price* 1.0006) - row["Trade open price"]) / row["Trade open price"])
                        # df_open_trade_info["PnL"] = df_open_trade_info["PnL"].apply(pd.to_numeric)
                        real_PnL = df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"].item()
                        balance = balance + real_PnL
                        frais = frais + ((row["Size"] + real_PnL) * 0.0004)
                        print(f'trade {row["Symbol"]} close withe PnL {real_PnL}')
                        nbr_open_trade -= 1

                    else :
                        actual = full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = actual
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"] = row["Size"] * -(((actual* 1.0006) - row["Trade open price"]) / row["Trade open price"])

                        if nbr_open_trade > 25 and row["PnL"] <= 0 :
                            df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trail stop price"]  = row["Trail stop price"] - ((row["Trail stop price"] - row["Actual price"]) / 3)


        engaged = df_open_trade_info.loc[df_open_trade_info['Status'] == 'Open', 'Size'].sum()
        # balance = balance - (engaged * 0.9994)
        df_open_trade_info = df_open_trade_info.tail(40)
        await asyncio.sleep(0.005)
        # print(df_open_trade_info)
        
    
    for key, value in occurences_count.items():

        if balance > 0 and nbr_open_trade < 20 and value >= 5 and len(full_rolling_dataframe) > 10 :

            trade_df = {}
            
            if full_rolling_dataframe[key + ' Var Last Price'].iloc[-1] > 0 and full_rolling_dataframe[key + ' Last Price'].iloc[-1] > full_rolling_dataframe[key + ' EMA_1'].iloc[-1] > full_rolling_dataframe[key + ' EMA_2'].iloc[-1]:

                trade_df = {"Timestamp": full_rolling_dataframe.index[-1],
                "Symbol": key,
                "Side" : "Long",
                "Size" : round(balance * 0.05, 2),
                "Trade open price": full_rolling_dataframe.iloc[-1][key + ' Last Price'],
                "Actual price" : full_rolling_dataframe.iloc[-1][key + ' Last Price'],
                "Trail stop price" : full_rolling_dataframe.iloc[-1][key + ' Last Price'] * (1 - trail_stop),
                "Status" : "Open",
                "Trade close price" : None,
                "Duration" : None,
                "PnL" : None}

            elif full_rolling_dataframe[key + ' Var Last Price'].iloc[-1] < 0 and full_rolling_dataframe[key + ' Last Price'].iloc[-1] < full_rolling_dataframe[key + ' EMA_1'].iloc[-1] < full_rolling_dataframe[key + ' EMA_2'].iloc[-1]: 

                trade_df = {"Timestamp": full_rolling_dataframe.index[-1],
                "Symbol": key,
                "Side" : "Short",
                "Size" : round(balance * 0.05, 2),
                "Trade open price": full_rolling_dataframe.iloc[-1][key + ' Last Price'],
                "Actual price" : full_rolling_dataframe.iloc[-1][key + ' Last Price'],
                "Trail stop price" : full_rolling_dataframe.iloc[-1][key + ' Last Price'] * (1 + trail_stop),
                "Status" : "Open",
                "Trade close price" : None,
                "Duration" : None,
                "PnL" : None}

            if trade_df is not None and not all(value is None for value in trade_df.values()):

                if df_open_trade_info.empty :
                    df_open_trade_info = pd.DataFrame([trade_df])
                    balance = balance - (trade_df["Size"] * 0.0004)
                    frais = frais + (trade_df["Size"] * 0.0004)
                    nbr_open_trade += 1

                elif key not in df_open_trade_info["Symbol"].values: 
                    newtradedfrow = pd.DataFrame([trade_df])
                    df_open_trade_info["Size"] = df_open_trade_info["Size"].apply(pd.to_numeric, errors='coerce')
                    df_open_trade_info = pd.concat([df_open_trade_info, newtradedfrow])
                    newtradedfrow = None
                    balance = balance - (trade_df["Size"] * 0.0004)
                    frais = frais + (trade_df["Size"] * 0.0004)
                    nbr_open_trade += 1

        else:
            continue
            
    df_open_trade_info_tail = df_open_trade_info.tail(40)
    clear_output(wait=True)
    display(full_rolling_dataframe.tail(10))
    display(df_open_trade_info_tail)
    display(f"nbr of open trade is {nbr_open_trade}")
    display(f"Balance is now : {balance}")
    display(f"Engage is now = {engaged}")
    display(f"Cumulated frees is now = {frais}")
    
            # pprint.pprint(trade_result)
            # pprint.pprint(f"nbr of open trade is {nbr_open_trade}")

    await asyncio.sleep(0.01)

In [4]:
# Initialiser un DataFrame vide pour stocker les données roulantes

full_rolling_dataframe = pd.DataFrame()
df_top_10 = pd.DataFrame()
df_open_trade_info = pd.DataFrame()
trade_result = pd.DataFrame()

balance = 1500
realbalance = 0
frais = 0
engaged = 0
trail_stop = 0.001
occurences_count = None
tickers_name_list = None
nbr_open_trade = 0

wind_Var_Price = 3
wind_EMA1 = 4
wind_Var_EMA1 = 3
wind_EMA2 = 9
wind_Var_EMA2 = 2
wind_Div_Price_EMA1 = 4
wind_Div_Moy_Price_EMA1 = 2
wind_Div_EMA1_2 = 6
wind_Div_Moy_EMA1_2 = 2

windparams = [wind_Var_Price, wind_EMA1 , wind_Var_EMA1, wind_EMA2, wind_Var_EMA2, wind_Div_Price_EMA1, wind_Div_Moy_Price_EMA1, wind_Div_EMA1_2, wind_Div_Moy_EMA1_2]

async def main():
    cvstest = "CSVTEST2_1000"
    try:
        nbr_open_trade = 0
        for x in range(10000):
            await fetch_data()
            await trad()           
            print(x)
        await exchange.close()
        # full_rolling_dataframe.to_csv(rf'C:\Users\IEL\Desktop\BitgetBotV1\{cvstest}.csv')
    except KeyboardInterrupt:
        await exchange.close()
# Exécutez la fonction main
await main()

,BTC Last Price,BTC Var Last Price,BTC Var Carre Last Price,BTC EMA_1,BTC Var EMA_1,BTC EMA_2,BTC Var EMA_2,BTC Diff Carre Price_EMA_1,BTC Divergence Price_EMA_1,BTC Divergence Moyen Price_EMA_1,...,PYTH Var EMA_1,PYTH EMA_2,PYTH Var EMA_2,PYTH Diff Carre Price_EMA_1,PYTH Divergence Price_EMA_1,PYTH Divergence Moyen Price_EMA_1,PYTH Diff Carre EMA_1_2,PYTH Divergence EMA_1_2,PYTH Divergence Moyen EMA_1_2,PYTH Condition
Timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-30 12:48:56.360,37851.1,0.004227,1.786980e-05,37851.062877,-0.000002,37852.530113,-0.000945,0.001378,-9.998118e+01,1.090214e+03,...,0.008470,0.410484,0.000966,3.823867e-10,326.481987,340.273984,1.365439e-11,-99.794673,-88.082580,0.0
2023-11-30 12:48:58.580,37851.1,0.000264,6.979837e-08,37851.077726,0.000105,37852.244091,-0.000756,0.000496,-9.997971e+01,2.967515e+02,...,0.005082,0.410487,0.000773,1.376592e-10,-95.334400,49.868395,9.116326e-13,-99.981658,-96.015299,0.0
2023-11-30 12:49:00.740,37851.1,0.000000,0.000000e+00,37851.086636,0.000063,37852.015272,-0.000605,0.000179,-8.771526e+01,4.044032e+01,...,0.003049,0.410490,0.000618,4.955732e-11,-95.334400,-46.933468,9.674594e-12,-99.727234,-98.489922,0.0
2023-11-30 12:49:02.930,37855.0,0.010304,1.061628e-04,37852.651982,0.004159,37852.612218,0.001577,5.513191,3.999627e+05,2.666553e+05,...,0.001829,0.410492,0.000495,1.784063e-11,-95.334400,-79.200756,1.518084e-11,-98.260078,-98.336693,0.0
2023-11-30 12:49:05.290,37858.9,0.020607,4.246510e-04,37855.151189,0.010738,37853.869774,0.003322,14.053585,2.832660e+06,1.977325e+06,...,0.001098,0.410494,0.000396,6.422628e-12,-95.334400,-89.956519,1.569563e-11,-90.370221,-93.025712,0.0
2023-11-30 12:49:07.520,37857.1,0.005547,3.077458e-05,37855.930713,0.008662,37854.515820,0.001707,1.367231,7.654298e+05,1.169395e+06,...,0.010403,0.410515,0.005189,3.784781e-09,7537.179181,4994.800614,5.605660e-10,4005.390901,2639.252030,1.0
2023-11-30 12:49:09.790,37859.9,0.002641,6.976924e-06,37857.518428,0.006253,37855.592656,0.002845,5.671885,2.878450e+00,3.898002e+05,...,0.015986,0.410532,0.004151,1.362521e-09,7537.179181,6689.719659,9.762585e-10,106989.024564,72205.767053,1.0
2023-11-30 12:49:12.450,37860.0,0.007660,5.868151e-05,37858.511057,0.006816,37856.474124,0.002329,2.216952,-8.422501e+01,1.298772e+05,...,0.009591,0.410545,0.003320,4.905076e-10,7537.179181,7254.692674,1.048372e-09,10736.343941,31226.151645,1.0
2023-11-30 12:49:14.640,37860.0,0.000264,6.976556e-08,37859.106634,0.004195,37857.179300,0.001863,0.798103,-4.162636e+01,4.326466e+04,...,0.005754,0.410556,0.002656,1.765827e-10,-95.334400,2354.674625,9.200510e-10,5960.606464,14382.454858,0.0


,Timestamp,Symbol,Side,Size,Trade open price,Actual price,Trail stop price,Status,Trade close price,Duration,PnL
0,2023-11-30 12:39:59.230,GAL,Short,75.00,1.56810,1.56960,1.568967,Close,1.5696,0 days 00:01:07,-0.116786
0,2023-11-30 12:40:01.290,GRT,Long,75.00,0.14322,0.14307,0.143127,Close,0.14307,0 days 00:00:41,-0.033598
0,2023-11-30 12:40:01.290,AR,Short,75.00,7.82700,7.83400,7.833826,Close,7.834,0 days 00:01:52,-0.112116
0,2023-11-30 12:40:01.290,KNC,Short,75.00,0.69420,0.69510,0.693893,Close,0.6951,0 days 00:02:20,-0.142293
0,2023-11-30 12:44:28.250,LPT,Long,74.97,6.39700,6.39900,6.395598,Open,None,None,0.068435
0,2023-11-30 12:45:50.450,MASK,Long,74.97,3.41900,3.41600,3.416580,Close,3.416,0 days 00:00:56,-0.020840


'nbr of open trade is 1'

'Balance is now : 1499.394392286576'

'Engage is now = 74.97'

'Cumulated frees is now = 0.32979374731463035'

263


C:\Users\IEL\AppData\Local\Temp\ipykernel_12632\3623069472.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Condition'] = (full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence Price_EMA_1'] > 0) & (full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Divergence EMA_1_2'] > 0)
C:\Users\IEL\AppData\Local\Temp\ipykernel_12632\3623069472.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  full_rolling_dataframe.loc[full_rolling_dataframe.index[-1], ticker_name + ' Condition'] = (full_rolling_dataframe.loc[full_rollin

CancelledError: 